**Imports**

In [1]:
# Imports
import pandas as pd
import random
import numpy as np

**Dataset imports**

In [2]:
train_df = pd.read_csv(r'training.csv')
test_df = pd.read_csv(r'testing.csv')
# Printing the column headings to better understand the features
print(train_df.columns.values)

['Duration' 'Distance' 'Pickup_longitude' 'Pickup_latitude'
 'Dropoff_latitude' 'Dropoff_latitude.1' 'Haversine' 'Pmonth' 'Pickup_day'
 'Pickup_hour' 'Pickup_minute' 'Pickup_weekday' 'Dropoff_month'
 'Dropoff_day' 'Dropoff_hour' 'Dropoff_minute' 'Dropoff_weekday' 'Temp'
 'Precip' 'Wind' 'Humid' 'Solar' 'Snow' 'GroundTemp' 'Dust']


In [15]:
len(train_df.columns.values)

25

In [3]:
from deap import gp, creator, base, tools, algorithms
import operator

In [17]:
def protectedDiv(x,y):
    try:
        return x/y
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 24)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin, pset=pset)

c:\Users\david\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\david\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [47]:
# Toolbox setup
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [93]:
def fitness_function(individual):
    print("evaluating fitness...")
    print("individual:")
    print(type(individual))
    trip_sample = train_df.sample(100, ignore_index=True, random_state=10)
    func = toolbox.compile(expr=individual)
    print("function compiled")
    fitness = 0
    for i in range(len(trip_sample)):
        print("fitness eval loop run " + str(i))
        # PSUEDOCODE: individual_fitness = abs(correct_duration - calculated_duration)
        # PSEUDOCODE: fitness = fitness + individual_fitness
        sample_row = trip_sample.iloc[[i]]
        print("sample row: " + str(type(sample_row)))
        sample_features = sample_row.drop("Duration",axis=1)
        print("sample features: " + str(type(sample_features)))

        correct_duration = trip_sample.at[i, "Duration"]
        print("correct duration for item " + str(i) + " is " + str(correct_duration))
        calculated_duration = func(sample_features)
        print("calculated duration for item " + str(i) + " is " + str(calculated_duration))

        individual_fitness = abs(correct_duration - calculated_duration)
        fitness = fitness + individual_fitness
    print("Fitness: " + str(fitness))
    fitness_tuple = (fitness)
    return fitness_tuple

In [94]:
toolbox.register("evaluate", fitness_function)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [92]:
pop = toolbox.population(n=100)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.5,
                                   halloffame=hof, verbose=True, ngen=2)

evaluating fitness...
individual:
<class 'deap.creator.Individual'>
function compiled
fitness eval loop run 0
sample row: <class 'pandas.core.frame.DataFrame'>
sample features: <class 'pandas.core.frame.DataFrame'>
correct duration for item 0 is 7


TypeError: <lambda>() missing 23 required positional arguments: 'ARG1', 'ARG2', 'ARG3', 'ARG4', 'ARG5', 'ARG6', 'ARG7', 'ARG8', 'ARG9', 'ARG10', 'ARG11', 'ARG12', 'ARG13', 'ARG14', 'ARG15', 'ARG16', 'ARG17', 'ARG18', 'ARG19', 'ARG20', 'ARG21', 'ARG22', and 'ARG23'